# Visualizing Tweets Using LDA Based Clustering

### Load Data

In [1]:
# Code to Import Libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords 
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
stopwords = set(stopwords.words('english'))
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
#nltk.download('stopwords')
#nltk.download('wordnet')

In [2]:
# Code Load_ Data into a List
tweetsInfo = pd.read_csv('AllTweetInfo.csv')

#Get Text for Topics
def MorePreprocessing(text):
    text_new = [t for t in text.split() if t not in stopwords]
    text_new = [t for t in text_new if not str.isnumeric(t)]
    
    return text_new
    
tweetsInfo['topic_text'] =tweetsInfo['text_features_new'].apply(MorePreprocessing)

### Creat LDA Model

In [3]:
# Train_LDA_ Model
t_list = tweetsInfo['topic_text'].tolist()
corpdict = corpora.Dictionary(t_list)

doc_term_matrix = [corpdict.doc2bow(doc) for doc in t_list]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = corpdict)


## Visualize Topics (Clusters)

In [4]:
# Visualize Models
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
corpora =  gensim.corpora.MmCorpus('corpus.mm')
vis =pyLDAvis.gensim.prepare(ldamodel, corpora, corpdict)
pyLDAvis.display(vis)

### Try to Find Right Setting

In [ ]:
# Code to FInd Right LDA Setting
#Refernece from https://markroxor.github.io/gensim/static/notebooks/gensim
def evaluate_graph(dictionary, corpus, texts, limit):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    for num_topics in range(1, limit):
        lm = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        lm_list.append(lm)
        cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
        c_v.append(cm.get_coherence())
        
    # Show graph
    x = range(1, limit)
    plt.plot(x, c_v)
    plt.xlabel("num_topics")
    plt.ylabel("Coherence score")
    plt.legend(("c_v"), loc='best')
    plt.show()
    
    return lm_list, c_v

lmlist, c_v = evaluate_graph(dictionary=corpdict, corpus=corpora, texts=t_list, limit=20)


In [9]:
pyLDAvis.gensim.prepare(lmlist[4], corpora, corpdict)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      22.718800        1       1  0.033450 -0.011423
1      21.217037        1       2  0.027553 -0.059803
0      19.065811        1       3 -0.080013  0.069475
2      18.652388        1       4 -0.064048 -0.050598
3      18.345970        1       5  0.083059  0.052349, topic_info=     Category        Freq            Term       Total  loglift  logprob
term                                                                   
250   Default   38.000000           offer   38.000000  30.0000  30.0000
456   Default  179.000000         quality  179.000000  29.0000  29.0000
2850  Default   57.000000             btc   57.000000  28.0000  28.0000
2528  Default   78.000000         bitcoin   78.000000  27.0000  27.0000
2109  Default   19.000000            gold   19.000000  26.0000  26.0000
2808  Default   36.000000  cryptocurrency   36.000000  25.0000  25.0000
2875  Default   15.000000               r   15.000000  24.0000  24.0000
15    Default  198.000000         service  198.000000  23.0000  23.0000
2727  Default   14.000000              oz   14.000000  22.0000  22.0000
2910  Default   15.000000       syndicate   15.000000  21.0000  21.0000
2851  Default   37.000000          crypto   37.000000  20.0000  20.0000
2906  Default   16.000000       freepicks   16.000000  19.0000  19.0000
2903  Default   16.000000      draftkings   16.000000  18.0000  18.0000
2020  Default   26.000000        ambiente   26.000000  17.0000  17.0000
2904  Default   16.000000         fanduel   16.000000  16.0000  16.0000
33    Default   40.000000             new   40.000000  15.0000  15.0000
2902  Default   16.000000             dfs   16.000000  14.0000  14.0000
2569  Default   18.000000             per   18.000000  13.0000  13.0000
2905  Default   18.000000           forex   18.000000  12.0000  12.0000
2811  Default   39.000000        ethereum   39.000000  11.0000  11.0000
117   Default   11.000000        gambling   11.000000  10.0000  10.0000
2799  Default   16.000000             buy   16.000000   9.0000   9.0000
476   Default   36.000000         product   36.000000   8.0000   8.0000
2913  Default   11.000000        verified   11.000000   7.0000   7.0000
223   Default   36.000000             day   36.000000   6.0000   6.0000
1597  Default   17.000000       venezuela   17.000000   5.0000   5.0000
2901  Default   12.000000          5dimes   12.000000   4.0000   4.0000
53    Default   27.000000            love   27.000000   3.0000   3.0000
2835  Default   21.000000        currency   21.000000   2.0000   2.0000
131   Default   12.000000            vega   12.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
5257   Topic5    2.731951     uttarakhand    4.275555   1.2479  -6.8725
1967   Topic5    1.954989     picoftheday    2.856524   1.3165  -7.2072
456    Topic5   49.925735         quality  179.245697   0.4175  -3.9670
5207   Topic5    2.330020        approves    3.582735   1.2655  -7.0317
4460   Topic5    1.952059       blackhawk    2.857064   1.3149  -7.2087
33     Topic5   14.767710             new   40.033020   0.6985  -5.1851
293    Topic5    7.028873            life   15.090288   0.9317  -5.9275
3432   Topic5    3.307852        handmade    5.757078   1.1416  -6.6813
53     Topic5   10.367691            love   27.990320   0.7026  -5.5389
5187   Topic5   16.771542     reservation   54.885818   0.5102  -5.0579
1321   Topic5   36.558651           price  196.564255   0.0137  -4.2786
15     Topic5   27.908009         service  198.899307  -0.2681  -4.5486
360    Topic5   10.195380               u   38.388458   0.3699  -5.5556
2729   Topic5    8.884586             usd   33.008160   0.3833  -5.6932
769    Topic5    4.271469            read    9.992751   0.8459  -6.4256
279    Topic5    4.835965       education   13.038876   0.7039  -6.3015
460    Topic5    4.758101         project   13.6894